In [1]:
import cv2
import numpy as np
import os
import math # For angle calculations
import time # For performance timing

# --- Configuration ---
# IMPORTANT: ADJUST THIS PATH to the folder containing your Carla environment images.
# Example: "C:/Users/YourUser/Documents/CarlaImages"
image_folder = "C:/Users/BCI-Lab/Downloads/teamA_dataset/_out_dataset/good_data" # <<< Adjust this path

# Supported image file extensions
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff"))]

print(f"Loaded {len(image_files)} images.")
if len(image_files) == 0:
    print("ERROR: No images found. Please check your 'image_folder' path or file extensions.")
    exit()

current_index = 0 # Start with the first image

# --- Create Tuning Window and Trackbars ---
cv2.namedWindow("Tuning", cv2.WINDOW_NORMAL)
# Set initial window size. You might need to maximize it manually for all views.
cv2.resizeWindow("Tuning", 1920, 1080)

print("\n--- Tuning Guide ---")
print("This version uses CLAHE (Contrast Limited Adaptive Histogram Equalization) for illumination normalization.")
print("The image is scaled down for processing to improve performance.")
print("First, focus on 'CLAHE Clip Limit' and 'CLAHE Tile Grid Size' to normalize image brightness.")
print("Then, adjust 'H_Lower', 'H_Upper', 'S_Lower', 'S_Upper', 'V_Lower', 'V_Upper' on the *CLAHE-processed* image.")
print("Observe 'CLAHE Processed Image', 'Color Mask (CLAHE+HSV)', and 'Final Mask' carefully.")
print("\n--- Performance Tips (if still laggy): ---")
print("  1. **Crucial:** Adjust 'Processing Scale (%)'. Smaller values mean faster processing.")
print("  2. Reduce 'Color Morph Kernel' and 'Edge Morph Kernel' sizes.")
print("  3. Consider setting 'Use Bilateral Filter' to 0 (off).")
print("  4. Reduce 'Display Height' in the code (around line ~350) for smaller preview windows.")
print("  5. Temporarily comment out some intermediate mask displays in `np.hstack` if needed.")
print("-------------------------------------")
print("Press 'd' or Right Arrow to go to the next image.")
print("Press 'a' or Left Arrow to go to the previous image.")
print("Press 's' to print current parameter values to the console.")
print("Press 'q' to quit.")


# --- HSV Color Space Sliders (for the target line detection AFTER Normalization) ---
cv2.createTrackbar("H_Lower", "Tuning", 5, 179, lambda x: None)
cv2.createTrackbar("H_Upper", "Tuning", 22, 179, lambda x: None)
cv2.createTrackbar("S_Lower", "Tuning", 100, 255, lambda x: None)
cv2.createTrackbar("S_Upper", "Tuning", 255, 255, lambda x: None)
cv2.createTrackbar("V_Lower", "Tuning", 100, 255, lambda x: None)
cv2.createTrackbar("V_Upper", "Tuning", 255, 255, lambda x: None)

# --- CLAHE (Contrast Limited Adaptive Histogram Equalization) Parameters ---
cv2.createTrackbar("CLAHE Clip Limit", "Tuning", 20, 100, lambda x: None) # Default 2.0 (scaled by 10)
cv2.createTrackbar("CLAHE Tile Grid Size", "Tuning", 8, 20, lambda x: None) # Default 8x8 (max 20x20)

# Processing Scale Trackbar
cv2.createTrackbar("Processing Scale (%)", "Tuning", 100, 100, lambda x: None) # 100% means original size, 50% means half size

# Morphological Kernel for Color Mask
cv2.createTrackbar("Color Morph Kernel", "Tuning", 5, 50, lambda x: None)

# Edge Pre-processing Filter
cv2.createTrackbar("Use Bilateral Filter", "Tuning", 1, 1, lambda x: None)
cv2.createTrackbar("Bilateral D", "Tuning", 9, 20, lambda x: None)
cv2.createTrackbar("Bilateral SigmaColor", "Tuning", 75, 200, lambda x: None)
cv2.createTrackbar("Bilateral SigmaSpace", "Tuning", 75, 200, lambda x: None)

# Canny Edge Detector Thresholds
cv2.createTrackbar("Canny Thresh1", "Tuning", 50, 255, lambda x: None)
cv2.createTrackbar("Canny Thresh2", "Tuning", 150, 255, lambda x: None)

# Morphological Kernel for Edge Mask
cv2.createTrackbar("Edge Morph Kernel", "Tuning", 7, 50, lambda x: None)

# Hough Line Transform Parameters
cv2.createTrackbar("Hough Threshold", "Tuning", 50, 200, lambda x: None)
cv2.createTrackbar("Hough Min Length", "Tuning", 30, 500, lambda x: None)
cv2.createTrackbar("Hough Max Gap", "Tuning", 50, 200, lambda x: None)

# Line Filtering Parameters
cv2.createTrackbar("Max Line Angle (deg)", "Tuning", 20, 90, lambda x: None)

# Image Cropping Parameters
cv2.createTrackbar("Crop % from Top", "Tuning", 75, 100, lambda x: None)
cv2.createTrackbar("Crop % from Left", "Tuning", 0, 49, lambda x: None)
cv2.createTrackbar("Crop % from Right", "Tuning", 0, 49, lambda x: None)


# --- Main Loop ---
while True:
    frame_start_time = time.time()
    filename = image_files[current_index]
    print(f"Processing: {filename} (Index: {current_index})")

    filepath = os.path.join(image_folder, filename)
    image = cv2.imread(filepath)
    if image is None:
        print(f"❌ ERROR: Cannot read {filename}. Skipping to next image.")
        current_index = (current_index + 1) % len(image_files)
        continue

    # --- Get current trackbar values ---
    # HSV
    lh = cv2.getTrackbarPos("H_Lower", "Tuning")
    uh = cv2.getTrackbarPos("H_Upper", "Tuning")
    ls = cv2.getTrackbarPos("S_Lower", "Tuning")
    us = cv2.getTrackbarPos("S_Upper", "Tuning")
    lv = cv2.getTrackbarPos("V_Lower", "Tuning")
    uv = cv2.getTrackbarPos("V_Upper", "Tuning")

    # CLAHE
    clahe_clip_limit = cv2.getTrackbarPos("CLAHE Clip Limit", "Tuning") / 10.0 # Scale from 1-100 to 0.1-10.0
    clahe_tile_grid_size = max(1, cv2.getTrackbarPos("CLAHE Tile Grid Size", "Tuning")) # Ensure at least 1x1

    processing_scale_percent = cv2.getTrackbarPos("Processing Scale (%)", "Tuning") / 100.0
    if processing_scale_percent <= 0: processing_scale_percent = 0.01 # Avoid division by zero/too small

    color_morph_kernel_size = max(1, cv2.getTrackbarPos("Color Morph Kernel", "Tuning"))
    
    use_bilateral = cv2.getTrackbarPos("Use Bilateral Filter", "Tuning")
    bilateral_d = max(1, cv2.getTrackbarPos("Bilateral D", "Tuning"))
    bilateral_sigma_color = max(1, cv2.getTrackbarPos("Bilateral SigmaColor", "Tuning"))
    bilateral_sigma_space = max(1, cv2.getTrackbarPos("Bilateral SigmaSpace", "Tuning"))

    canny_thresh1 = cv2.getTrackbarPos("Canny Thresh1", "Tuning")
    canny_thresh2 = cv2.getTrackbarPos("Canny Thresh2", "Tuning")
    edge_morph_kernel_size = max(1, cv2.getTrackbarPos("Edge Morph Kernel", "Tuning"))
    
    hough_threshold = cv2.getTrackbarPos("Hough Threshold", "Tuning")
    hough_min_length = cv2.getTrackbarPos("Hough Min Length", "Tuning")
    hough_max_gap = cv2.getTrackbarPos("Hough Max Gap", "Tuning")

    max_line_angle_deg = cv2.getTrackbarPos("Max Line Angle (deg)", "Tuning")
    
    crop_percent_top = cv2.getTrackbarPos("Crop % from Top", "Tuning")
    crop_percent_left = cv2.getTrackbarPos("Crop % from Left", "Tuning")
    crop_percent_right = cv2.getTrackbarPos("Crop % from Right", "Tuning")

    # --- 1. Apply Cropping ---
    crop_start_time = time.time()
    original_h, original_w, _ = image.shape

    crop_y_start = int(original_h * crop_percent_top / 100)
    if crop_y_start >= original_h:
        crop_y_start = original_h - 1
    
    crop_x_start = int(original_w * crop_percent_left / 100)
    crop_x_end = original_w - int(original_w * crop_percent_right / 100)

    if crop_x_start >= crop_x_end:
        crop_x_start = 0
        crop_x_end = original_w

    cropped_image = image[crop_y_start:original_h, crop_x_start:crop_x_end].copy()
    cropped_h, cropped_w, _ = cropped_image.shape
    print(f"  Cropping: {(time.time() - crop_start_time):.4f}s")

    # --- 2. Resize for Processing Performance ---
    processing_width = int(cropped_w * processing_scale_percent)
    processing_height = int(cropped_h * processing_scale_percent)
    
    # Ensure minimum size to avoid errors
    if processing_width < 1: processing_width = 1
    if processing_height < 1: processing_height = 1

    preprocess_resize_start_time = time.time()
    cropped_image_for_processing = cv2.resize(cropped_image, (processing_width, processing_height), interpolation=cv2.INTER_LINEAR)
    print(f"  Pre-processing Resize: {(time.time() - preprocess_resize_start_time):.4f}s")


    # --- 3. Illumination Normalization (CLAHE) ---
    clahe_start_time = time.time()
    
    # Convert to HSV color space
    hsv_image = cv2.cvtColor(cropped_image_for_processing, cv2.COLOR_BGR2HSV)
    
    # Split the HSV image into H, S, and V channels
    h, s, v = cv2.split(hsv_image)
    
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clahe_clip_limit, tileGridSize=(clahe_tile_grid_size, clahe_tile_grid_size))
    
    # Apply CLAHE to the V-channel
    v_clahe = clahe.apply(v)
    
    # Merge the enhanced V-channel back with H and S channels
    normalized_hsv = cv2.merge([h, s, v_clahe])
    
    # Convert back to BGR
    normalized_image = cv2.cvtColor(normalized_hsv, cv2.COLOR_HSV2BGR)
    print(f"  CLAHE Processing: {(time.time() - clahe_start_time):.4f}s")

    # --- 4. Color Masking (HSV on CLAHE-processed image) ---
    color_mask_start_time = time.time()
    hsv_normalized = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2HSV)
    
    lower_hsv = np.array([lh, ls, lv])
    upper_hsv = np.array([uh, us, uv])
    color_mask = cv2.inRange(hsv_normalized, lower_hsv, upper_hsv)
    print(f"  Color Masking: {(time.time() - color_mask_start_time):.4f}s")

    # --- 5. Morphological Operations on the Color Mask ---
    morph_color_start_time = time.time()
    color_morph_kernel = np.ones((color_morph_kernel_size, color_morph_kernel_size), np.uint8)
    # Use MORPH_OPEN to remove small objects/noise
    color_mask_morphed = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, color_morph_kernel, iterations=1)
    # Dilate to connect broken parts
    color_mask_morphed = cv2.dilate(color_mask_morphed, color_morph_kernel, iterations=1)
    # Use MORPH_CLOSE to close small holes and connect nearby components
    color_mask_morphed = cv2.morphologyEx(color_mask_morphed, cv2.MORPH_CLOSE, color_morph_kernel, iterations=1)
    print(f"  Color Morph: {(time.time() - morph_color_start_time):.4f}s")
    
    # --- 6. Edge Detection (Canny) ---
    edge_detect_start_time = time.time()
    gray_normalized = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2GRAY)
    
    if use_bilateral:
        filtered_gray = cv2.bilateralFilter(gray_normalized, bilateral_d, bilateral_sigma_color, bilateral_sigma_space)
    else:
        # Using a small default Gaussian blur if bilateral is off
        filtered_gray = cv2.GaussianBlur(gray_normalized, (5, 5), 0) 

    edge_mask = cv2.Canny(filtered_gray, canny_thresh1, canny_thresh2)

    edge_morph_kernel = np.ones((edge_morph_kernel_size, edge_morph_kernel_size), np.uint8)
    edge_mask_morphed = cv2.morphologyEx(edge_mask, cv2.MORPH_CLOSE, edge_morph_kernel, iterations=1)
    print(f"  Edge Detect & Morph: {(time.time() - edge_detect_start_time):.4f}s")

    # --- 7. Combine Masks and Line Detection ---
    line_detection_start_time = time.time()
    final_mask = cv2.bitwise_and(color_mask_morphed, edge_mask_morphed)

    lines = cv2.HoughLinesP(final_mask, 1, np.pi / 180,
                            hough_threshold,
                            minLineLength=hough_min_length,
                            maxLineGap=hough_max_gap)

    # --- 8. Prepare Preview and Calculate Deviation Score ---
    preview = image.copy()
    # Draw ROI rectangle on original image
    cv2.rectangle(preview, (crop_x_start, crop_y_start), (crop_x_end, original_h), (255, 0, 255), 2)
    # Draw center line on original image
    center_x_full_img = original_w // 2
    cv2.line(preview, (center_x_full_img, crop_y_start), (center_x_full_img, original_h), (0, 255, 255), 2)

    deviation_score = float('nan')
    display_text = "NO LINE"
    cx_full_img = -1 

    filtered_lines = []
    if lines is not None:
        # Scale lines back to original cropped_image dimensions for drawing
        # (This is important because `final_mask` was created from `cropped_image_for_processing`)
        scale_x = cropped_w / processing_width
        scale_y = cropped_h / processing_height
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            
            # Scale coordinates back to original cropped image size
            x1_orig = int(x1 * scale_x)
            y1_orig = int(y1 * scale_y)
            x2_orig = int(x2 * scale_x)
            y2_orig = int(y2 * scale_y)

            # Calculate angle of the line
            angle_rad = math.atan2(y2_orig - y1_orig, x2_orig - x1_orig)
            angle_deg = math.degrees(angle_rad)

            # Filter lines by angle (e.g., to keep mostly horizontal or near-horizontal lines)
            if abs(angle_deg) < max_line_angle_deg or abs(angle_deg - 180) < max_line_angle_deg:
                filtered_lines.append((x1_orig, y1_orig, x2_orig, y2_orig)) # Store original-scaled coords

    if filtered_lines:
        all_line_midpoints_x = []
        for x1, y1, x2, y2 in filtered_lines:
            cv2.line(preview,
                     (x1 + crop_x_start, y1 + crop_y_start), # Add crop_x_start to shift to full image coords
                     (x2 + crop_x_start, y2 + crop_y_start), # Add crop_x_start to shift to full image coords
                     (0, 255, 0), 2) # Green color for detected lines
            all_line_midpoints_x.append((x1 + x2) // 2)
        
        # Calculate the average X-coordinate of all filtered line midpoints within the cropped area
        cx_cropped_scaled = int(np.mean(all_line_midpoints_x))
        # Convert this X-coordinate back to the full original image's coordinate system
        cx_full_img = cx_cropped_scaled + crop_x_start
        # Y-coordinate for the centroid visualization (arbitrary, usually center of ROI)
        cy_full_img = crop_y_start + (original_h - crop_y_start) // 2 

        # Draw centroid and its coordinate on the preview image
        cv2.circle(preview, (cx_full_img, cy_full_img), 6, (0, 0, 255), -1) # Red circle
        cv2.putText(preview, f"cx: {cx_full_img}", (cx_full_img + 10, cy_full_img), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Calculate deviation score
        if center_x_full_img != 0:
            deviation_score = (cx_full_img - center_x_full_img) / center_x_full_img
        else:
            deviation_score = 0.0 # Handle case where center_x_full_img is 0 (shouldn't happen for typical images)

        display_text = f"Deviation: {deviation_score:.2f}"
    
    cv2.putText(preview, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2) # Yellow text
    print(f"  Line Detection & Draw: {(time.time() - line_detection_start_time):.4f}s")

    # --- 9. Display images with consistent, larger size ---
    display_start_time = time.time()
    # You can reduce this further for more speed. Try 200 or 150 if still too slow.
    display_height = 300 
    
    # Calculate display width based on original image's aspect ratio
    aspect_ratio_orig = original_w / original_h
    display_width = int(display_height * aspect_ratio_orig)

    preview_resized = cv2.resize(preview, (display_width, display_height))
    cropped_image_resized = cv2.resize(cropped_image, (display_width, display_height))
    
    # Normalized image was processed at 'processing_width' x 'processing_height', resize to display size
    normalized_display_resized = cv2.resize(normalized_image, (display_width, display_height))

    # Convert masks to BGR for display purposes (so they show up as grayscale images)
    color_mask_resized = cv2.resize(cv2.cvtColor(color_mask, cv2.COLOR_GRAY2BGR), (display_width, display_height))
    morphed_mask_resized = cv2.resize(cv2.cvtColor(color_mask_morphed, cv2.COLOR_GRAY2BGR), (display_width, display_height))
    edge_mask_morphed_resized = cv2.resize(cv2.cvtColor(edge_mask_morphed, cv2.COLOR_GRAY2BGR), (display_width, display_height))
    final_mask_resized = cv2.resize(cv2.cvtColor(final_mask, cv2.COLOR_GRAY2BGR), (display_width, display_height))

    # Stack all images horizontally for display
    combined_display = np.hstack((
        preview_resized,
        cropped_image_resized,
        normalized_display_resized, # Now displays CLAHE processed image
        color_mask_resized,
        morphed_mask_resized,
        edge_mask_morphed_resized,
        final_mask_resized
    ))

    cv2.imshow("Tuning", combined_display)
    print(f"  Display Update: {(time.time() - display_start_time):.4f}s")
    print(f"Total frame time: {(time.time() - frame_start_time):.4f}s\n")

    # --- Keyboard Controls ---
    key = cv2.waitKey(30) & 0xFF # Wait 30ms for a key press
    if key == ord("q"): # Press 'q' to quit
        break
    elif key in [ord("d"), 83]: # Press 'd' or Right Arrow to go to the next image (ASCII for Right Arrow)
        current_index = (current_index + 1) % len(image_files)
    elif key in [ord("a"), 81]: # Press 'a' or Left Arrow to go to the previous image (ASCII for Left Arrow)
        current_index = (current_index - 1 + len(image_files)) % len(image_files)
    elif key == ord("s"): # Press 's' to print current parameters to console
        print(f"\n--- Saved Parameters for {filename} ---")
        print(f"hsv_lower = np.array([{lh}, {ls}, {lv}])")
        print(f"hsv_upper = np.array([{uh}, {us}, {uv}])")
        print(f"clahe_clip_limit = {clahe_clip_limit}")
        print(f"clahe_tile_grid_size = {clahe_tile_grid_size}")
        print(f"processing_scale_percent = {processing_scale_percent}")
        print(f"color_morph_kernel_size = {color_morph_kernel_size}")
        print(f"use_bilateral_filter = {use_bilateral}")
        print(f"bilateral_d = {bilateral_d}")
        print(f"bilateral_sigma_color = {bilateral_sigma_color}")
        print(f"bilateral_sigma_space = {bilateral_sigma_space}")
        print(f"canny_thresh1 = {canny_thresh1}")
        print(f"canny_thresh2 = {canny_thresh2}")
        print(f"edge_morph_kernel_size = {edge_morph_kernel_size}")
        print(f"hough_threshold = {hough_threshold}")
        print(f"hough_min_length = {hough_min_length}")
        print(f"hough_max_gap = {hough_max_gap}")
        print(f"max_line_angle_deg = {max_line_angle_deg}")
        print(f"crop_percent_top = {crop_percent_top}")
        print(f"crop_percent_left = {crop_percent_left}")
        print(f"crop_percent_right = {crop_percent_right}")
        
        if math.isnan(deviation_score):
            print(f"Calculated Deviation: NO LINE (cx={cx_full_img})\n")
        else:
            print(f"Calculated Deviation: {deviation_score:.2f} (cx={cx_full_img})\n")

cv2.destroyAllWindows()

Loaded 1291 images.

--- Tuning Guide ---
This version uses CLAHE (Contrast Limited Adaptive Histogram Equalization) for illumination normalization.
The image is scaled down for processing to improve performance.
First, focus on 'CLAHE Clip Limit' and 'CLAHE Tile Grid Size' to normalize image brightness.
Then, adjust 'H_Lower', 'H_Upper', 'S_Lower', 'S_Upper', 'V_Lower', 'V_Upper' on the *CLAHE-processed* image.
Observe 'CLAHE Processed Image', 'Color Mask (CLAHE+HSV)', and 'Final Mask' carefully.

--- Performance Tips (if still laggy): ---
  1. **Crucial:** Adjust 'Processing Scale (%)'. Smaller values mean faster processing.
  2. Reduce 'Color Morph Kernel' and 'Edge Morph Kernel' sizes.
  3. Consider setting 'Use Bilateral Filter' to 0 (off).
  4. Reduce 'Display Height' in the code (around line ~350) for smaller preview windows.
  5. Temporarily comment out some intermediate mask displays in `np.hstack` if needed.
-------------------------------------
Press 'd' or Right Arrow to go

KeyboardInterrupt: 